In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [2]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [3]:
img_folder = r'D:\original'
out_folder = r'D:\ToAWS2'

course_list = ['청도','포항','중문','시흥','거창','속리산','춘천']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
  '시흥':'MGC004',
  '거창':'MGC005',
  '속리산':'MGC006',
  '춘천':'MGC007'
}

folder_type = ['rgb','ndvi','thumb']

In [4]:
course_id = course_ids.get('거창')
course_id

'MGC005'

In [5]:
target_folders = glob.glob(os.path.join(out_folder,'**/*'))
target_folders = [x for x in target_folders if course_id in x]
target_folders

['D:\\ToAWS2\\MGC005\\20230809',
 'D:\\ToAWS2\\MGC005\\20230817',
 'D:\\ToAWS2\\MGC005\\20230906',
 'D:\\ToAWS2\\MGC005\\20230927']

In [6]:
target_folder = target_folders[0]

In [39]:
target_date = os.path.split(target_folder)[-1]
course_id, target_date

('MGC005', '20230906')

In [62]:
dataJson_ = glob.glob(os.path.join(out_folder,course_id,target_date,'**/photo*.json'), recursive=True)[0]
dataJson_ 

'D:\\ToAWS2\\MGC005\\20230906\\photo.json'

In [63]:
with open(dataJson_, "r") as _json:
  dataJson= json.load(_json)

In [64]:
len(dataJson)

8866

In [65]:
DataJson4Test = []

for index_, row_ in enumerate(dataJson):
  newrow = {}
  newrow ['id'] = row_['id']
  newrow ['metadata'] = course_id+target_date+row_['info']['area']+row_['info']['desc']
  newrow ['date'] = target_date
  newrow ['area'] = row_['info']['area']
  newrow ['desc'] = row_['info']['desc']
  newrow ['json'] = json.loads(json.dumps(row_), parse_float=Decimal)
  DataJson4Test.append(newrow)

In [66]:
df = pd.DataFrame(DataJson4Test)

In [67]:
df.groupby(['date','area','desc','metadata']).agg({'id':"nunique"}).reset_index()

,date,area,desc,metadata,id
0,20230906,거창 클럽디 EAST 1,미기재_0,MGC00520230906거창 클럽디 EAST 1미기재_0,78
1,20230906,거창 클럽디 EAST 1,"티,켄터키,KB블루그래스_0","MGC00520230906거창 클럽디 EAST 1티,켄터키,KB블루그래스_0",501
2,20230906,거창 클럽디 EAST 1,"티,켄터키,KB블루그래스_1","MGC00520230906거창 클럽디 EAST 1티,켄터키,KB블루그래스_1",501
3,20230906,거창 클럽디 EAST 1,"티,켄터키,KB블루그래스_2","MGC00520230906거창 클럽디 EAST 1티,켄터키,KB블루그래스_2",501
4,20230906,거창 클럽디 EAST 1,"티,켄터키,KB블루그래스_3","MGC00520230906거창 클럽디 EAST 1티,켄터키,KB블루그래스_3",163
5,20230906,거창 클럽디 EAST 2,미기재_0,MGC00520230906거창 클럽디 EAST 2미기재_0,95
6,20230906,거창 클럽디 EAST 2,"이종잔디,톨페스큐_0","MGC00520230906거창 클럽디 EAST 2이종잔디,톨페스큐_0",377
7,20230906,거창 클럽디 EAST 2,"이종잔디,페레니얼라이그래스_0","MGC00520230906거창 클럽디 EAST 2이종잔디,페레니얼라이그래스_0",501
8,20230906,거창 클럽디 EAST 2,"이종잔디,페레니얼라이그래스_1","MGC00520230906거창 클럽디 EAST 2이종잔디,페레니얼라이그래스_1",501
9,20230906,거창 클럽디 EAST 2,"이종잔디,페레니얼라이그래스_2","MGC00520230906거창 클럽디 EAST 2이종잔디,페레니얼라이그래스_2",101


In [68]:
import boto3
from decimal import Decimal
# Get the service resource.
dynamodb = boto3.resource('dynamodb', region_name='ap-northeast-2')
table = dynamodb.Table('dsnia_imagelabelsV2')
# table = dynamodb.Table('dsnia_imagelabels')

In [75]:
target_jsons = [x for x in DataJson4Test if x['metadata'] == "MGC00520230906거창 클럽디 EAST 1티,켄터키,KB블루그래스_0"]
len(target_jsons)

501

In [76]:
with table.batch_writer(overwrite_by_pkeys=['metadata', 'id']) as batch:
  for index_, row_ in enumerate(target_jsons):
    # print(index_, row_['id'], end='\r')
    batch.put_item(
      Item= row_
    )

In [105]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

lastEvaluatedKey = None
items = [] # Result Array

while True:
    if lastEvaluatedKey == None:
        response = table.scan(FilterExpression=Attr('metadata').eq("MGC00520230906거창 클럽디 EAST 1티,켄터키,KB블루그래스_0")) # This only runs the first time - provide no ExclusiveStartKey initially
    else:
        response = table.scan(
        FilterExpression=Attr('metadata').eq("MGC00520230906거창 클럽디 EAST 1티,켄터키,KB블루그래스_0"),
        ExclusiveStartKey=lastEvaluatedKey # In subsequent calls, provide the ExclusiveStartKey
    )

    items.extend(response['Items']) # Appending to our resultset list
    

    # Set our lastEvlauatedKey to the value for next operation,
    # else, there's no more results and we can exit
    if 'LastEvaluatedKey' in response:
        lastEvaluatedKey = response['LastEvaluatedKey']
    else:
        break

print(len(items)) # Return Value: 6

501


In [10]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

courseid = "MGC005"
date = "20230906"

area = "거창 클럽디 EAST 1"

desc = "티,켄터키,KB블루그래스_0"

response = table.query(KeyConditionExpression=Key('metadata').eq("MGC00520230906거창 클럽디 EAST 1티,켄터키,KB블루그래스_1"))
data =  response['Items']

while 'LastEvaluatedKey' in response:
    response = table.query(KeyConditionExpression=Key('metadata').eq("MGC00520230906거창 클럽디 EAST 1티,켄터키,KB블루그래스_1"),  ExclusiveStartKey=response['LastEvaluatedKey']  )    
    # response_ = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

In [39]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

courseid = "MGC005"
date = "20230906"

area = "거창 클럽디 EAST 1"

desc = "티,켄터키,KB블루그래스_0"

response = table.query(KeyConditionExpression=Key('metadata').eq(courseid+date+area+desc))
data =  response['Items']

while 'LastEvaluatedKey' in response:
    response = table.query(KeyConditionExpression=Key('metadata').eq(courseid+date+area+desc),  ExclusiveStartKey=response['LastEvaluatedKey']  )    
    # response_ = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

In [40]:
len(data)

501

In [41]:
response = table.query(
    KeyConditionExpression=Key('metadata').eq(courseid+date+area+desc),    
)
data = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

In [42]:
len(data)

59777

In [12]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError



response = table.scan()
data = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

In [13]:
len(data)

168870

In [7]:
import boto3
from decimal import Decimal
# Get the service resource.
dynamodb = boto3.resource('dynamodb', region_name='ap-northeast-2')
table = dynamodb.Table('dsnia_imagelabelsV2')
# table = dynamodb.Table('dsnia_imagelabels')

In [45]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

courseid = "MGC001"
date = "20230906"

area = "거창 클럽디 EAST 1"

desc = "티,켄터키,KB블루그래스_0"

response = table.scan(
    IndexName='id-courseid-index',
    FilterExpression=Attr('courseid').eq(courseid)
)

data = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(
      # IndexName='id-courseid-index',
      # FilterExpression=Attr('courseid').eq(courseid),
      ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

In [46]:
len(data)

0